MOT_bbox_to_json

In [ ]:
import json
import os
# without cv2 ver
def mot_to_json(input_file):
    # Create a dictionary to store the JSON structure
    result = {}

    with open(input_file, 'r') as f:
        for line in f:
            # Split and parse the line
            frame, obj_id, bb_left, bb_top, bb_width, bb_height, _, _, _, _ = line.strip().split(',')
            frame = str(int(frame)).zfill(4)  # Format frame number as 4-digit string
            obj_id = str(int(obj_id))

            # Create bounding box in the [left, top, right, bottom] format
            bbox = [
                float(bb_left),
                float(bb_top),
                float(bb_left) + float(bb_width),
                float(bb_top) + float(bb_height)
            ]

            # Insert data into the result dictionary
            if frame not in result:
                result[frame] = {"person": {}, "car": {}}
            result[frame]["person"][obj_id] = bbox

    # Write the result to a JSON file
    output_file = "bbox_" + input_file.split('/')[-1].replace('.txt', '.json')
    with open(output_file, 'w') as jf:
        json.dump(result, jf, indent=4)

    print(f"Saved result to {output_file}")
    return output_file

# Example usage
input_file = 'car_txt_archive/car_b1c9c847-3bda4659.txt'
mot_to_json(input_file)


### Task
now, I want to improve current mot_to_json function

#### modified MOT 2D challenge format

The file format contain one object instance per line. Each line must contain 10 values:

<frame>, <id>, <bb_left>, <bb_top>, <bb_left>, <bb_bottom>, <conf>, <class_id>, <-1>, <-1>
The bbox is xyxy format
The conf value contains the detection confidence in the det.txt files.
The class_id value indicate the type of detected object by YOLO. # class_ids of interest - person 0, car 2, motorcycle 3, bus 5 and truck 7

### code
import json
import os

def mot_to_json(input_file):
    # Create a dictionary to store the JSON structure
    result = {}

    with open(input_file, 'r') as f:
        for line in f:
            # Split and parse the line
            frame, obj_id, bb_left, bb_top, bb_width, bb_height, _, _, _, _ = line.strip().split(',') # need to be modified 
            frame = str(int(frame)).zfill(4)  # Format frame number as 4-digit string
            obj_id = str(int(obj_id))

            # Create bounding box in the [left, top, right, bottom] format
            bbox = [
                # need to be fill again 
            ]

            # Insert data into the result dictionary
            if frame not in result:
                result[frame] = {"person": {}, "car": {}}
            result[frame]["person"][obj_id] = bbox

    # Write the result to a JSON file
    output_file = "bbox_" + input_file.split('/')[-1].replace('.txt', '.json')
    with open(output_file, 'w') as jf:
        json.dump(result, jf, indent=4)

    print(f"Saved result to {output_file}")
    return output_file

# Example usage
input_file = 'car_txt_archive/car_b1c9c847-3bda4659.txt'
mot_to_json(input_file)


In [ ]:
import json
import os

def mot_to_json(input_file):
    # Create a dictionary to store the JSON structure
    result = {}

    # Dictionary to map class IDs to their corresponding names
    class_id_to_name = {0: "person", 2: "car", 3: "motorcycle", 5: "bus", 7: "truck"}

    with open(input_file, 'r') as f:
        for line in f:
            # Split and parse the line
            frame, obj_id, bb_left, bb_top, bb_right, bb_bottom, conf, class_id, _, _ = line.strip().split(',')
            frame = str(int(frame)).zfill(4)  # Format frame number as 4-digit string
            obj_id = str(int(obj_id))
            class_name = class_id_to_name[int(class_id)]  # Convert class_id to class name

            # Create bounding box in the [left, top, right, bottom] format
            bbox = [float(bb_left), float(bb_top), float(bb_right), float(bb_bottom)]

            # Insert data into the result dictionary
            if frame not in result:
                result[frame] = {"person": {}, "car": {}, "motorcycle": {}, "bus": {}, "truck": {}}
            result[frame][class_name][obj_id] = bbox

    # Write the result to a JSON file
    output_file = os.path.join(os.path.dirname(input_file), "bbox_" + os.path.basename(input_file).replace('.txt', '.json'))
    with open(output_file, 'w') as jf:
        json.dump(result, jf, indent=4)

    print(f"Saved result to {output_file}")
    return output_file

# Example usage
input_file = 'car_txt_archive/car_b1c9c847-3bda4659.txt'
output_file = mot_to_json(input_file)


In [ ]:
import json
def fill_missing_frames(json_file_path):

   # json_file_path = "JSON_data/" + json_file_path
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Convert the frame keys to integers, sort them, and then find the min and max frame numbers.
    frame_numbers = sorted([int(frame) for frame in data.keys()])
    if not frame_numbers:
        print(f"no detected obj in current video:{json_file_path}")
        return
    min_frame, max_frame = frame_numbers[0], frame_numbers[-1]

    # Create an empty frame structure
    empty_frame = {
        "person": {},
        "car": {}
    }

    # Fill in the missing frames with the empty structure
    for frame in range(min_frame, max_frame + 1):
        frame_str = str(frame).zfill(4)  # Convert frame number back to zero-padded string
        if frame_str not in data:
            data[frame_str] = empty_frame

    with open(json_file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"finished filling frames for {json_file_path}")
    
fill_missing_frames(output_file)

In [ ]:

def process_txt_archive(txt_folder):
    # Iterate over all files in the txt_archive folder
    for filename in os.listdir(txt_folder):
        if filename.endswith(".txt"):
            input_file = os.path.join(txt_folder, filename)

            output_file = mot_to_json(input_file)
            
            fill_missing_frames(output_file)

process_txt_archive('txt_archive')

In [ ]:
import cv2
import json
import os
def mot_to_json(input_file, video_path,output_dir):
    # Create a dictionary to store the JSON structure
    result = {}

    # Dictionary to map class IDs to their corresponding names
    class_id_to_name = {0: "person", 2: "car", 3: "motorcycle", 5: "bus", 7: "truck"}

    with open(input_file, 'r') as f:
        for line in f:
            # Split and parse the line
            frame, obj_id, bb_left, bb_top, bb_right, bb_bottom, conf, class_id, _, _ = line.strip().split(',')
            frame = str(int(frame)).zfill(4)  # Format frame number as 4-digit string
            obj_id = str(int(obj_id))
            class_name = class_id_to_name[int(class_id)]  # Convert class_id to class name

            # Create bounding box in the [left, top, right, bottom] format
            bbox = [float(bb_left), float(bb_top), float(bb_right), float(bb_bottom)]

            # Insert data into the result dictionary
            if frame not in result:
                result[frame] = {"person": {}, "car": {}, "motorcycle": {}, "bus": {}, "truck": {}}
            result[frame][class_name][obj_id] = bbox

    # Create an empty frame structure
    empty_frame = {
        "person": {}, 
        "car": {},
        "motorcycle": {},
        "bus": {},
        "truck": {}
    }

    # Get the total number of frames in the video
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    # Fill the result with empty frames if they don't exist
    for frame_num in range(total_frames):
        frame = str(frame_num).zfill(4)
        if frame not in result:
            result[frame] = empty_frame

    # Write the result to a JSON file
    output_file = os.path.join(os.path.dirname(output_dir), "bbox_" + os.path.basename(input_file).replace('.txt', '.json'))
    with open(output_file, 'w') as jf:
        json.dump(result, jf, indent=4)

    print(f"Saved result to {output_file}")
    return output_file

# Example usage
input_file = './car_videos/car_b1c9c847-3bda4659.txt'
video_path = '../videos/bdd/b1c9c847-3bda4659.mp4'  # Make sure to specify the actual path to your video
output_dir = './json/'
mot_to_json(input_file, video_path, output_dir)


In [ ]:
def process_txt_archive(txt_folder, video_folder, output_dir):
    """
    Process all .txt files in the txt_folder.

    Args:
        txt_folder (str): Path to the folder containing .txt files.
        video_folder (str): Path to the folder containing corresponding video files.
        output_dir (str): Directory path for JSON results.

    Returns:
        None
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all files in the txt_folder
    for filename in os.listdir(txt_folder):
        if filename.endswith(".txt"):
            input_file = os.path.join(txt_folder, filename)
            
            # Extract video name from the txt filename by removing the prefix "car_" and the .txt extension
            video_name = filename[len("car_"):].rsplit('.', 1)[0]


            # Assuming the video could be in .mp4 or .mov format
            video_path_mp4 = os.path.join(video_folder, video_name + '.mp4')
            video_path_mov = os.path.join(video_folder, video_name + '.mov')
            video_path = video_path_mp4 if os.path.exists(video_path_mp4) else video_path_mov
            
            if not os.path.exists(video_path):
                print(f"Warning: Corresponding video for {filename} not found. Skipping.")
                continue

            mot_to_json(input_file, video_path, output_dir)

# Specify paths
txt_folder = 'txt_archive'
video_folder = '../videos/jaad'  # Adjust this to where your videos are
output_dir = './json/bdd'
process_txt_archive(txt_folder, video_folder, output_dir)

